<a href="https://colab.research.google.com/github/nehaa28/llm/blob/main/llama3_SMSText_FineTuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()

!pip install "unsloth[colab-new] @git+https://github.com/unslothai/unsloth.git"

if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["Instruction"]
    inputs       = examples["Input"]
    outputs      = examples["Output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt_format.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = load_dataset("HugOsa/SmsBillText", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.

    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/6 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.469800
2,0.857100
3,1.599400
4,1.423900
5,0.780200
6,1.756800
7,1.488200
8,0.477500
9,0.616100
10,0.554200


In [ ]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    prompt_format.format(
        "Continue the fibonnaci sequence.",
        "1, 1, 2, 3, 5, 8",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13<|end_of_text|>']

In [ ]:
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context.
                   Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    prompt_format.format(
        "Extract Bill information for this text.",
        "Your credit card bill is due on 15-Nov. Total amount due is $136.54. Minimum amount is $9.00",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nExtract Bill information for this text.\n\n### Input:\nYour credit card bill is due on 15-Nov. Total amount due is $136.54. Minimum amount is $9.00\n\n### Response:\nBill: Yes, Amount: $136.54, Due Date: 15-Nov. Pas<|end_of_text|>']

In [ ]:
def Extract_Bill(text):
 inputs = tokenizer(
 [
    prompt_format.format(
        "Extract Bill information for this text.",
        text,
        "",)
 ], return_tensors = "pt").to("cuda")

 outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
 o = tokenizer.batch_decode(outputs)[0]

 return o[o.find("### Response")+14:len(o)-1]

In [ ]:
s = r[0].find("### Response")
l = len(r[0])


'### Response:\nBill: Yes, Amount: 300.15, Due Date: 10-March<|end_of_text|'

In [ ]:
VALIDATION_DATA = [
    ("The total amount due is 300, and it should be paid by 10-May."),
    ("Your Credit Card bill amount due is 300.15, Min Amount Due is 25.00, pay by 10/10."),
    ("Your bill amnt_due is 390.15, and due_date is 36/14."),
    ("Your Credit Card has been disabled on 12/10."),
    ("Your Credit Card is now active."),
    ("This is some spam text."),
    ("You paid 300.55 using your credit card on 10-Feb at Food Mart."),
    ("Welcome to New Delhi. Enjoying Roaming service on Airtel"),
]

for text in VALIDATION_DATA:
    response = Extract_Bill(text)
    print(f"Text: {text} **** Response {response}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Text: The total amount due is 300, and it should be paid by 10-May. **** Response Bill: Yes, Amount: 300.00, Due Date: 10-March<|end_of_text|


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Text: Your Credit Card bill amount due is 300.15, Min Amount Due is 25.00, pay by 10/10. **** Response Bill: Yes, Amount: 300.15, Due Date: 10-March<|end_of_text|


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Text: Your Credit Card is now active. **** Response Bill: No<|end_of_text|


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Text: This is some spam text. **** Response Bill: No<|end_of_text|


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Text: You paid 300.55 using your credit card on 10-Feb at Food Mart. **** Response Bill: No<|end_of_text|
Text: Welcome to New Delhi. Enjoying Roaming service on Airtel **** Response Bill: No<|end_of_text|
